In [5]:
import numpy as np
import pydae.build as db
import sympy as sym

In [6]:
Omega_b = 2*np.pi*50
L_t = 1.25e-3; #Converter side Filter Inductance H
R_t = L_t*Omega_b/10; #Converter side Filter resistance Ohms. Quality Factor of 10
L_s = 1.25e-3; #Grid Side Filter Inductance H
R_s = L_s*Omega_b/10; #Grid side Filter resistance Ohms. Quality Factor of 10
R_d = 20; #Dmaping resistor
C_m = 4e-6; #Filter capacitance
tau_ctrl_1 = 5e-3; #Time constant of CTRL 1
Ts_Power=50e-6; #Sample time of power plant
Ts_Control=50e-6; #Sample time of controller
K_pi = (L_t+L_s)/tau_ctrl_1; #Proportional gain of CTRL 1
K_ii = (R_t+R_s)/tau_ctrl_1; #Integral gain of CTRL 1
H = 5.0; # desired virtual inertia 
K_p = 0.01; # active power proportinal gain
#H = T_p/K_p/2

T_p = K_p*2*H;  # active power integral time constant

K_q = 0.1; # reactive power proportinal gain
T_q = 0.1; # reactive power integral time constant

X_v = 0.05; # virtual reactance
R_v = 0.01;  # virtual resistance

K_pv = 0.0004
K_iv = 11.313708

g01 = {"name":"g01","bus": "1","U_b":400.0,"S_b_kVA":30,"v_dc":750,'type':'sm2_i',
           'R_t':R_t,'L_t':L_t,'R_s':R_s,'L_s':L_s,'C_m':C_m,'K_phi':0.00,'omega':Omega_b, 'G_d':1.0, #VSC
           'T_eta':0.01,   # CTRL0
           'K_pi':K_pi,'K_ii':K_ii,'Gv_in':0.1,   # CTRL1
           'K_pv':K_pv,'K_iv':K_iv,   # CTRL2
           'K_p':K_p,'T_p':T_p,'K_q':K_q,'T_q':T_q,'R_v':R_v,'X_v':X_v,'p_m_ref':0.5,'q_s_ref':0.2,   # CTRL3
           'H':5.0,'D':5.0,  # CTRL3
           'K_f':20.0, 'K_vr':20.0}   # CTRL4

data = {
"system":{"f_hz":50,"model_type":"ae"},
"buses": [{"bus":"1"},{"bus":"2"}],
"lines":     [  {"bus_j":"1", "bus_k":"2", "R":0.01, "X":0.1, "B":1e-3}
             ],
"loads" : [
    #    {"bus": "R06", "kVA": 55.0, "pf": 0.95, "T_i":0.01,"I_max":1200}
        ],
"grid_formers" : [
                  {"bus": "1","V_phph":400.0, "deg":0.0},
                  {"bus": "2","V_phph":400.0, "deg":0.0},
                 ],
"vsgs" : [g01]}


In [7]:
cos = sym.cos
sin = sym.sin

vsg = g01

name = vsg['name']
bus = vsg['bus']

U_b = vsg['U_b']
S_b = vsg['S_b_kVA']*1000
I_b = S_b/(np.sqrt(3)*U_b)

U_bdq = U_b*(np.sqrt(2))
V_bdq = U_bdq/np.sqrt(3)
I_bdq = I_b*np.sqrt(2)

eta_D,eta_Q =sym.symbols(f'eta_D_{name},eta_Q_{name}',real=True)

params_vsg = {}
u_vsg = {}

## Transformations #########################################################################
## feedbacks:
feedbacks = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ'] + ['phi']
for item in feedbacks:
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    

v_md,v_mq = sym.symbols(f'v_md_{name},v_mq_{name}', real=True)
v_sd,v_sq = sym.symbols(f'v_sd_{name},v_sq_{name}', real=True)
i_sd,i_sq = sym.symbols(f'i_sd_{name},i_sq_{name}', real=True)
i_td,i_tq = sym.symbols(f'i_td_{name},i_tq_{name}', real=True)
eta_d,eta_q = sym.symbols(f'eta_d_{name},eta_q_{name}', real=True)
phi,dum= sym.symbols(f'phi_{name},dum_{name}', real=True)

eq_v_md = -v_md + v_mD*cos(phi) + v_mQ*sin(phi) # original park
eq_v_mq = -v_mq - v_mD*sin(phi) + v_mQ*cos(phi) # original park

eq_v_sd = -v_sd + v_sD*cos(phi) + v_sQ*sin(phi) # original park
eq_v_sq = -v_sq - v_sD*sin(phi) + v_sQ*cos(phi) # original park

eq_i_sd = -i_sd + i_sD*cos(phi) + i_sQ*sin(phi) # original park
eq_i_sq = -i_sq - i_sD*sin(phi) + i_sQ*cos(phi) # original park

# jmm: medimos i_t?
eq_i_td = -i_td + i_tD*cos(phi) + i_tQ*sin(phi) # original park
eq_i_tq = -i_tq - i_tD*sin(phi) + i_tQ*cos(phi) # original park

eq_eta_D = -eta_D + eta_d*cos(phi) - eta_q*sin(phi) # original park
eq_eta_Q = -eta_Q + eta_d*sin(phi) + eta_q*cos(phi) # original park

g_aux = [eq_v_md,eq_v_mq,eq_v_sd,eq_v_sq,eq_i_td,eq_i_tq,eq_i_sd,eq_i_sq,eq_eta_D,eq_eta_Q]    
y_aux = [   v_md,   v_mq,   v_sd,   v_sq,   i_td,   i_tq,   i_sd,   i_sq,   eta_D,   eta_Q]
y_0_aux = [  0.0,  V_bdq,    0.0,  V_bdq,       0,      0]    
        
        
## parameters:
params_vsc_filter = {}
for item in ['L_t','R_t','C_m','L_s','R_s','omega','G_d']:
    params_vsc_filter.update({f'{item}_{name}':vsg[item]})
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
## inputs:
u_vsc_filter = {}
for item in ['v_dc']:#,'eta_D','eta_Q']:
    u_vsc_filter.update({f'{item}_{name}':vsg[item]})
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
## dynamic states:
x_list_vsc_filter = []
for item in x_list_vsc_filter:
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
## algebraic states
y_list_vsc_filter = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ']
for item in y_list_vsc_filter:
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    
## feedbacks:
v_poiD,v_poiQ = sym.symbols(f'v_{bus}_D,v_{bus}_Q', real=True)
i_poiD,i_poiQ = sym.symbols(f'i_{bus}_D,i_{bus}_Q', real=True)

#eta_D = eta_D_ref # - Gv_in*(i_tD - i_sD)
#eta_Q = eta_Q_ref #- Gv_in*(i_tQ - i_sQ)

# LCL filter
di_tD = 1/L_t*(eta_D/2*v_dc - R_t*i_tD + omega*L_t*i_tQ - v_mD)  
di_tQ = 1/L_t*(eta_Q/2*v_dc - R_t*i_tQ - omega*L_t*i_tD - v_mQ) 
dv_mD = 1/C_m*(i_tD + C_m*omega*v_mQ - G_d*v_mD - i_sD) 
dv_mQ = 1/C_m*(i_tQ - C_m*omega*v_mD - G_d*v_mQ - i_sQ) 
di_sD = 1/L_s*(v_mD - R_s*i_sD + omega*L_s*i_sQ - v_sD)  
di_sQ = 1/L_s*(v_mQ - R_s*i_sQ - omega*L_s*i_sD - v_sQ) 

# Grid interaction
eq_i_poiD =  i_sD - i_poiD
eq_i_poiQ =  i_sQ - i_poiQ   

eq_v_sD =  v_sD - v_poiD
eq_v_sQ =  v_sQ - v_poiQ    

#grid_dae['params'].pop(f'i_{bus}_D')
#grid_dae['params'].pop(f'i_{bus}_Q')

# DAE
f_vsc_filter = [di_tD, di_tQ, dv_mD, dv_mQ, di_sD, di_sQ]
x_vsc_filter = [ i_tD,  i_tQ,  v_mD,  v_mQ,  i_sD,  i_sQ]
g_vsc_filter = [ eq_i_poiD, eq_i_poiQ, eq_v_sD, eq_v_sQ]
y_vsc_filter = [    i_poiD,    i_poiQ,    v_sD,    v_sQ ]    
x_0_vsc_filter = [ ]
y_0_vsc_filter = [ 0.0,  0.0,   0.0, V_bdq,     0,     0,         0,         0,       0,   V_bdq]


## Model integration
f_vsg = f_vsc_filter 
x_vsg = x_vsc_filter
g_vsg = g_vsc_filter + g_aux
y_vsg = y_vsc_filter + y_aux
params_vsg.update(params_vsc_filter)
#params_vsg.update(params_ctrl_1)
u_vsg.update(u_vsc_filter)
#u_vsg.update(u_ctrl_0)
#u_vsg.update(u_ctrl_1)   
u_vsg.update({'v_1_D':0.0,'v_1_Q':326,'phi_g01':0.0,'eta_d_g01':0.0,'eta_q_g01':0.8})
#u_vsg.update({'v_1_D':0.0,'v_1_Q':326,'eta_d_g01':0.0,'eta_q_g01':0.8})

In [9]:
i_sd_g01,i_sq_g01 = sym.symbols(f'i_sd_g01,i_sq_g01', real=True)
sys_dict = {'name':'vsc_lcl',
           'params_dict':params_vsg,
           'f_list':[f_vsg],
           'g_list':[g_vsg],
           'x_list':[x_vsg],
           'y_ini_list':y_vsg,
           'y_run_list':y_vsg,
           'u_run_dict':u_vsg,
           'u_ini_dict':u_vsg,
           'h_dict':{'v_t_D':eta_D/2*v_dc,'v_t_Q':eta_Q/2*v_dc,'damp_D':G_d*v_mD,'damp_Q':G_d*v_mQ,
                     'i_sd_g01':i_sd,'i_sq_g01':i_sq,
                     'v_md_g01':v_md,'v_mq_g01':v_mq,
                     'i_td_g01':i_td,'i_tq_g01':i_tq,
                     'v_sd_g01':v_sd,'v_sq_g01':v_sq,}
           }

db.system(sys_dict)
db.sys2num(sys_dict)

In [11]:
u_vsg

{'v_dc_g01': 750,
 'v_1_D': 0.0,
 'v_1_Q': 326,
 'phi_g01': 0.0,
 'eta_d_g01': 0.0,
 'eta_q_g01': 0.8}

In [6]:
f_vsg

[(L_t_g01*i_tQ_g01*omega_g01 - R_t_g01*i_tD_g01 + eta_D_g01*v_dc_g01/2 - v_mD_g01)/L_t_g01,
 (-L_t_g01*i_tD_g01*omega_g01 - R_t_g01*i_tQ_g01 + eta_Q_g01*v_dc_g01/2 - v_mQ_g01)/L_t_g01,
 (C_m_g01*omega_g01*v_mQ_g01 - G_d_g01*v_mD_g01 - i_sD_g01 + i_tD_g01)/C_m_g01,
 (-C_m_g01*omega_g01*v_mD_g01 - G_d_g01*v_mQ_g01 - i_sQ_g01 + i_tQ_g01)/C_m_g01,
 (L_s_g01*i_sQ_g01*omega_g01 - R_s_g01*i_sD_g01 + v_mD_g01 - v_sD_g01)/L_s_g01,
 (-L_s_g01*i_sD_g01*omega_g01 - R_s_g01*i_sQ_g01 + v_mQ_g01 - v_sQ_g01)/L_s_g01]

In [7]:
sys_dict['g'].jacobian(sys_dict['u_run'])

Matrix([
[0,  0,  0,                                                0,            0,             0],
[0,  0,  0,                                                0,            0,             0],
[0, -1,  0,                                                0,            0,             0],
[0,  0, -1,                                                0,            0,             0],
[0,  0,  0,   -v_mD_g01*sin(phi_g01) + v_mQ_g01*cos(phi_g01),            0,             0],
[0,  0,  0,   -v_mD_g01*cos(phi_g01) - v_mQ_g01*sin(phi_g01),            0,             0],
[0,  0,  0,   -v_sD_g01*sin(phi_g01) + v_sQ_g01*cos(phi_g01),            0,             0],
[0,  0,  0,   -v_sD_g01*cos(phi_g01) - v_sQ_g01*sin(phi_g01),            0,             0],
[0,  0,  0,   -i_tD_g01*sin(phi_g01) + i_tQ_g01*cos(phi_g01),            0,             0],
[0,  0,  0,   -i_tD_g01*cos(phi_g01) - i_tQ_g01*sin(phi_g01),            0,             0],
[0,  0,  0,   -i_sD_g01*sin(phi_g01) + i_sQ_g01*cos(phi_g01),          

In [9]:
sys_dict['x']

Matrix([
[i_tD_g01],
[i_tQ_g01],
[v_mD_g01],
[v_mQ_g01],
[i_sD_g01],
[i_sQ_g01]])